# Neeps- Medium

In [1]:
import findspark
import pandas as pd
findspark.init()

SVR = '192.168.31.31'
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

sc = (SparkSession.builder.appName('app15-2') 
      .master(f'spark://{SVR}:7077') 
      .config('spark.sql.warehouse.dir', f'hdfs://{SVR}:9000/user/hive/warehouse') 
      .config('spark.cores.max', '4') 
      .config('spark.executor.instances', '1') 
      .config('spark.executor.cores', '2') 
      .config('spark.executor.memory', '10g') 
      .enableHiveSupport().getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
ut_staff = sc.read.table('sqlzoo.ut_staff')
ut_student = sc.read.table('sqlzoo.ut_student')
ut_event = sc.read.table('sqlzoo.ut_event')
ut_room = sc.read.table('sqlzoo.ut_room')
ut_attends = sc.read.table('sqlzoo.ut_attends')
ut_teaches = sc.read.table('sqlzoo.ut_teaches')
ut_occurs = sc.read.table('sqlzoo.ut_occurs')
ut_modle = sc.read.table('sqlzoo.ut_modle')
ut_week = sc.read.table('sqlzoo.ut_week')

## 6.
**Show the 'size' of each of the co72010 events. Size is the total number of students attending each event.**

In [3]:
(ut_event.withColumnRenamed('id', 'event')
 .filter(ut_event['modle']=='co72010')
 .join(ut_attends, on='event')
 .join(ut_student, on=(ut_attends['student']==ut_student['id']))
 .groupBy('event')
 .sum('sze')
 .orderBy(col('sum(sze)').desc())
 .toPandas())

,event,sum(sze)
0,co72013.L01,215
1,co72013.T02,49
2,co72013.T04,45
3,co72013.T01,40
4,co72013.T05,35
5,co72013.T06,29
6,co72013.T03,10
7,co72013.L02,10


## 7.
**For each post-graduate module, show the size of the teaching team. (post graduate modules start with the code co7).**

In [4]:
(ut_teaches
 .join(ut_event.withColumnRenamed('id', 'event'), on='event')
 .join(ut_modle.filter(ut_modle['id'].like('co7%')), 
       on=(ut_event['modle']==ut_modle['id']))
 .select('id', 'staff')
 .distinct()
 .groupBy('id')
 .agg(count('staff'))
 .toPandas())

,id,count(staff)
0,co72003,2
1,co72011,2
2,co72018,1
3,co72017,1
4,co72026,2
5,co72016,2
6,co72033,1
7,co72010,2
8,co72006,1
9,co72021,1


## 8.
**Give the full name of those modules which include events taught for fewer than 10 weeks.**

In [5]:
(ut_modle.withColumnRenamed('name', 'modle_name')
 .join(ut_event.withColumnRenamed('id', 'event'), 
       on=(ut_modle['id']==ut_event['modle']))
 .join(ut_occurs, on='event')
 .groupBy('event', 'modle_name')
 .count()
 .orderBy('modle_name')
 .filter(col('count')<10)
 .select('modle_name')
 .distinct()
 .toPandas())

,modle_name
0,Interactivity and the Internet
1,Internet Multimedia
2,Project
3,Languages and Algorithms


## 9.
**Identify those events which start at the same time as one of the co72010 lectures.**

In [6]:
t = (ut_event
     .join(ut_occurs, on=(ut_event['id']==ut_occurs['event']))
     .withColumn('time', concat_ws('', col('week'), col('dow'), col('tod'))))
(t.filter(col('modle') != 'co72010')
 .join(t.filter(col('modle') == 'co72010')
       .select('time'), on='time')
 .select('id')
 .distinct()
 .orderBy('id')
 .toPandas())

,id
0,co12004.T04
1,co12004.T05
2,co12005.T01
3,co12005.T04
4,co12006.L03
5,co12008.L01
6,co12012.T01
7,co22005.T02
8,co22005.T04
9,co22005.T07


## 10.
**How many members of staff have contact time which is greater than the average?**

In [7]:
t = (ut_event.join(ut_teaches, on=(ut_event['id']==ut_teaches['event']))
     .join(ut_occurs, on=(ut_event['id']==ut_occurs['event']))
     .groupBy('staff')
     .agg(sum('duration').alias('duration')))

(t.filter(col('duration') > t.groupBy().sum('duration').collect()[0][0] /
          ut_teaches.select('staff').distinct().count())
 .groupBy()
 .count()
 .toPandas())

,count
0,19


In [8]:
sc.stop()